In [1]:
import numpy as np
import pandas as pd

def load_data():
    train = pd.read_csv('titanic/train.csv')
    test = pd.read_csv('titanic/test.csv')
    submission = pd.read_csv('titanic/gender_submission.csv')

    test = test.merge(submission, on='PassengerId')

    def preprocess(df):
        df = df.copy()
        df['Age'] = df['Age'].fillna(df['Age'].median())
        df['Fare'] = df['Fare'].fillna(df['Fare'].median())
        df['Embarked'] = df['Embarked'].fillna('S')

        df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
        df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
        df['FamilySize'] = df['SibSp'] + df['Parch']
        return df

    train = preprocess(train)
    test = preprocess(test)

    features = ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize']
    X_train = train[features].values
    y_train = train['Survived'].values
    X_test = test[features].values
    y_test = test['Survived'].values

    return X_train, y_train, X_test, y_test